# LightGBM v1

### Imports

In [18]:
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

### Funciones

In [19]:
def normalizar_texto(descripcion):
    
    descripcion = descripcion.replace(',', ' ')
    descripcion = descripcion.replace('.', ' ')
    descripcion = descripcion.replace('<', ' ')
    descripcion = descripcion.replace('>', ' ')
    descripcion = descripcion.replace('\n', ' ')
    descripcion = descripcion.replace('\strong', ' ')
    descripcion = descripcion.replace('\bold', ' ')
    descripcion = descripcion.lower()
    
    return descripcion

def contar_palabras(titulo, palabras):
    
    contadas = 0
    
    for palabra in palabras:
        spliteado = titulo.split()
        contadas += spliteado.count(palabra)
        
    return contadas

### Reads
train.csv -> train
<br>
test.csv -> test

In [106]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

### Preparo el modelo 

**Feature Direción:**

In [21]:
DIRINVALIDAS = set(['Calle','domicilio conocido','calle','CALLE','--','Sin nombre ','1','.','-','x #x','0','...','S/N'])

df = train

df.loc[df.direccion.isin(DIRINVALIDAS), 'direccion'] = 0
df.loc[df['direccion'] != 0, 'direccion'] = 1

df['direccion'].value_counts()

1    237759
0      2241
Name: direccion, dtype: int64

**Feature Titulo y Descripcion:**

In [22]:
df['titulo'] = df['titulo'].fillna(value = ".")
df['titulo'] = df['titulo'].apply(normalizar_texto)
df['descripcion'] = df['descripcion'].fillna(value = ".")
df['descripcion'] = df['descripcion'].apply(normalizar_texto)

df['calefaccion'] = df['titulo'].apply(contar_palabras, palabras=["calefaccion", "calefacción", "calefaccionado", "aireacondicinado",
                    "acondicionado", "estufa","chimenea"]) + df['descripcion'].apply(contar_palabras, palabras=["calefaccion", 
                    "calefacción", "calefaccionado", "aireacondicinado", "acondicionado", "estufa","chimenea"])

df['suite'] = df['titulo'].apply(contar_palabras, palabras=["suite"])+df['descripcion'].apply(contar_palabras, palabras=["suite"])

df['avenida'] = df['titulo'].apply(contar_palabras, palabras=["avenida", "av", "avenidas", "bulevar", 
               "boulevard", "paseo", "vía"])+df['descripcion'].apply(contar_palabras, palabras=["avenida", "av", 
                "avenidas", "bulevar","boulevard", "paseo", "vía"])

df['gim'] = df['titulo'].apply(contar_palabras, palabras=["gimnasio", "gimnásio", "entrenamiento", "gim", "gym", "fit",
            "ejercicio", "gimnasia","atletismo", "cancha"])+df['descripcion'].apply(contar_palabras, palabras=["gimnasio",
            "gimnásio", "entrenamiento", "gim", "gym", "fit","ejercicio", "gimnasia","atletismo", "cancha"])

df['cochera'] = df['titulo'].apply(contar_palabras, palabras=["cochera", "cocheras", "garage", 
                "garages", "garaje", "garajes"])+df['descripcion'].apply(contar_palabras, palabras=["cochera", 
                "cocheras", "garage","garages", "garaje", "garajes"])

df['ubicacion'] = df['titulo'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                "centrica", "céntrico", "céntrica","central"])+df['descripcion'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                "centrica", "céntrico", "céntrica","central"])

df['balcon'] = df['titulo'].apply(contar_palabras, palabras=["balcon", "balcón", "terraza", "palco", "mirador", 
            "balconcillo","azotea", "solana"])+df['descripcion'].apply(contar_palabras, palabras=["balcon", "balcón", 
            "terraza", "palco", "mirador", "balconcillo","azotea", "solana"])

df['camaraseg'] = df['titulo'].apply(contar_palabras, palabras=["camara", "cámara", "cámaras", "camaras","seguridad",
                "guardia","seguro","protegido"]) + df['descripcion'].apply(contar_palabras, palabras=["camara", "cámara",
                "cámaras", "camaras","seguridad","guardia","seguro","protegido"])

df['parque'] = df['titulo'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", "glorieta","jardin",
            "jardín","patio"])+df['descripcion'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", 
            "glorieta","jardin","jardín","patio"])

df['amoblado'] = df['titulo'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble","decorado",
                "listo"]) + df['descripcion'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble",
                "decorado","listo"])

df['bañera'] = df['titulo'].apply(contar_palabras, palabras=["bañera", "hidromasaje", "hidro", "tina",
            "jacuzzi","jacuzi","yacuzi"]) + df['descripcion'].apply(contar_palabras, palabras=["bañera", "hidromasaje", 
            "hidro", "tina","jacuzzi","jacuzi","yacuzi"])


df['estreno'] = df['titulo'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
            "innovador"]) + df['descripcion'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
            "innovador"])

df['transporte'] = df['titulo'].apply(contar_palabras, palabras=["subte", "subterraneo", "subterráneo","metro", 
                "estacion", "estación", "tren","subestacion", "subestación", "ferrocarril","metrobús", "metrobus", 
                "trolebus","trolebús", "bus", "bús"]) + df['descripcion'].apply(contar_palabras, palabras=["subte", 
                "subterraneo", "subterráneo","metro", "estacion", "estación", "tren","subestacion", "subestación", 
                "ferrocarril","metrobús", "metrobus", "trolebus","trolebús", "bus", "bús"])

df['pileta'] = df['titulo'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])+ df['descripcion'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])
                                       
df['lujo'] = df['titulo'].apply(contar_palabras, palabras=["lujo", "delujo", "deluxe", "delúxe", "lujosa", "lujoso", 
            "lujosas", "lujosos","exclusivo","vip"]) + df['descripcion'].apply(contar_palabras, palabras=["lujo", 
            "delujo","deluxe", "delúxe", "lujosa", "lujoso", "lujosas", "lujosos","exclusivo","vip"])

df['humilde'] = df['titulo'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
            "económico", "económica", "barata", "barato", "accesible", "baratillo",
            "baratilla", "rebajado", "ganga", "asequible", "módico", "módica","credito","crédito","oferta","oferton",
            "imperdible"]) + df['descripcion'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
            "económico", "económica", "barata", "barato", "accesible", "baratillo",
            "baratilla", "rebajado", "ganga", "asequible", "módico", "módica",
            "credito","crédito","oferta","oferton","imperdible"]) 

df['ventana'] = df['titulo'].apply(contar_palabras, palabras=["ventana", "ventanas", 
            "vista", "ventanal","vistas","cristal"]) + df['descripcion'].apply(contar_palabras, palabras=["ventana",
            "ventanas", "vista", "ventanal","vistas","cristal"])

df['nuevo'] = df['titulo'].apply(contar_palabras, palabras=["reciente", "recien", "recién", "nueva", "nuevo", "nuevas", 
            "nuevos", "estrenar"]) + df['descripcion'].apply(contar_palabras, palabras=["reciente", "recien", "recién", 
            "nueva", "nuevo", "nuevas", "nuevos", "estrenar"])

df['luz'] = df['titulo'].apply(contar_palabras, palabras=["luz", "luminoso", "luminosa","claridad", "luminiscencia", 
        "luminosidad", "iluminación","iluminacion"]) + df['descripcion'].apply(contar_palabras, palabras=["luz", 
        "luminoso", "luminosa","claridad", "luminiscencia","luminosidad", "iluminación","iluminacion"])

df['bueno'] = df['titulo'].apply(contar_palabras, palabras=["bueno", "buena", "buenas", "buenos","excelente", 
            "excelentes","increible","espectacular"]) + df['descripcion'].apply(contar_palabras, palabras=["bueno", 
            "buena", "buenas", "buenos","excelente", "excelentes","increible","espectacular"])

df['contable'] = df['titulo'].apply(contar_palabras, palabras=["precio"]) + df['descripcion'].apply(contar_palabras, palabras=["precio"])

#Nuevos Feat

df['agente'] = df['descripcion'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"]) \
             + df['titulo'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"])

df['garante'] = df['descripcion'].apply(contar_palabras, palabras=["garante", "garantía", "fiador", "garantizador", "avalista", "garantia",
                "defensor", "garantías", "garantes", "codeudor"]) + df['titulo'].apply(contar_palabras, palabras=["garante", 
                "garantía", "fiador", "garantizador", "avalista", "garantia", "defensor", "garantías", "garantes", "codeudor"])

df['finanza'] =  df['descripcion'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario"\
                 ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocréditp", "ahorro"]) + \
                 df['titulo'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario" \
                 ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocrédito", "ahorro"]) 

df['turismo'] =  df['descripcion'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                   "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])\
                 + df['titulo'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                   "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])

df['longitud_desc'] =  df['descripcion'].apply(lambda x: len(x.split()))

#Nuevas de V4

df['longitud_titulo'] = df['titulo'].apply(lambda x: len(x))

df['cant_!'] = df['titulo'].apply(lambda x: x.count('!')) 

df['tranquilidad'] = df['descripcion'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                        "paz", "calma", "calmo", "quietud"])\
                 + df['titulo'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                        "paz", "calma", "calmo", "quietud"])

df['reparacion'] = df['descripcion'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                            "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada",
                                                                     "refaccion","refacción","refacciones"])\
                 + df['titulo'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                            "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada"
                                                                     "refaccion","refacción","refacciones"])

df['mascotas'] = df['descripcion'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                "gatos", "gato", "gata", "animal", "animales"])\
                 + df['titulo'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                "gatos", "gato", "gata", "animal", "animales"])

df['accesibilidad'] = df['descripcion'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])\
                 + df['titulo'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])

df['normas'] =  df['descripcion'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                "ilegal", "legal", "multa", "infraccion", "infracción"])\
                 + df['titulo'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                "ilegal", "legal", "multa", "infraccion", "infracción"])

df['beneficios'] = df['descripcion'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                "agregado", "gratuito", "gratuitamente", \
                                                "regalo"])\
                 + df['titulo'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                "agregado", "gratuito", "gratuitamente", \
                                                "regalo"])

df['conexion'] = df['descripcion'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                   "señal"])\
                 + df['titulo'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                   "señal"])

df['servicios_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["servicio", "servicios"])

df['metros_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["metros","m2"])

df['acabados'] = df['descripcion'].apply(contar_palabras, palabras = ["acabados","acabado","terminacion","terminación"])

df['plusvalia'] = df['descripcion'].apply(contar_palabras, palabras = ["plusvalia", "plusvalía"])

#Nuevos de V5

df['cocina'] = df['descripcion'].apply(contar_palabras, palabras=['cocina', 'cocinas'])

df['alberca'] = df['descripcion'].apply(contar_palabras, palabras=['alberca'])

df['negacion'] = df['descripcion'].apply(contar_palabras, palabras = ["no"])

df['variospisos'] = df['descripcion'].apply(contar_palabras, palabras = ["escalera","escaleras",
                    "ascensor", "elevador", "escalinata", "gradas", "escalerilla"])

df['vestidor'] = df['descripcion'].apply(contar_palabras, palabras = ["vestidor"])

**Feature Metros:**

In [23]:
df['metrosdescubiertos'] = abs(df['metrostotales']-df['metroscubiertos'])
df['relacionmetros'] = df['metrostotales']/df['metroscubiertos']

In [24]:
def mi_split (df):

    X = df.drop(['precio'], axis = 1)
    Y = df['precio']
    x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.35)
    return x_train, x_test, y_train, y_test

from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn import preprocessing

def minmax(serie):
    
    return (serie-serie.min())/(serie.max()-serie.min())

def mean_target_encoding(train, nombrefeat, nombretarget):
    
    temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean)
    train[nombrefeat + "_mean"]=(temp-temp.min())/(temp.max()-temp.min())
    
    return train

def mean_target_decoding(x_test, nombrefeat, x_train):
    
    nombrefeatmean = nombrefeat + "_mean"
    
    temp = x_train.loc[:,[nombrefeat,nombrefeatmean]]
    temp = temp.set_index(nombrefeat)
    temp = temp.drop_duplicates()
    temp = temp.T.squeeze()
    values = x_test[nombrefeat].map(temp)
    x_test[nombrefeatmean] = values 

    return x_test

def armar_set_2(train):
    
    #Puedo resolver de forma general las que son iguales para train y test
    #es decir, aquellas que no filtran informacion a los de validacion
    
    train = train.drop(['descripcion', 'titulo', 'direccion', 'id'], axis = 1)
    
    #Adiciono servicios
    train['servicios'] = train['piscina'] + train['gimnasio'] + train['usosmultiples']
    
    train['fecha'] = pd.to_datetime(train['fecha']).dt.year
    
    #"Normalizo" la antiguedad
    #train['antiguedad'] = minmax(train['antiguedad'])
    
    #Elimino los residuos
    #train = train.drop(['piscina', 'gimnasio','usosmultiples','escuelascercanas', 
                       # 'centroscomercialescercanos'], axis = 1)
    
    #Hago el split                    
    x_train, x_test, y_train, y_test = mi_split(train)
        
    x_train["precio"] = y_train
    x_test["precio"] = y_test
    
    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometro']=df1['precio']/df1['metrostotales']

    PMM = df1.groupby('idzona').agg({'preciometro': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometro": "precioMetroMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')


    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometroCub']=df1['precio']/df1['metroscubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroCub': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroCub": "precioMetroCubMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')
    
    
    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometroDes']=df1['precio']/df1['metrosdescubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroDes': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroDes": "precioMetroDesMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')


    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometro']=df1['precio']/df1['metrostotales']
    
    PMM = df1.groupby('ciudad').agg({'preciometro': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometro": "precioMetroMeanC"})

    x_train = x_train.merge(PMM, on='ciudad',how='left')
    x_test = x_test.merge(PMM, on='ciudad',how='left')
    
    
    df1 = x_train.dropna(subset=['metroscubiertos'])
    df1['preciometroCub']=df1['precio']/df1['metroscubiertos']

    PMM = df1.groupby('ciudad').agg({'preciometroCub': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroCub": "precioMetroCubMeanC"})

    x_train = x_train.merge(PMM, on='ciudad',how='left')
    x_test = x_test.merge(PMM, on='ciudad',how='left')
    
    
    df1 = x_train.dropna(subset=['metrosdescubiertos'])
    df1['preciometroDes']=df1['precio']/df1['metrosdescubiertos']

    PMM = df1.groupby('ciudad').agg({'preciometroDes': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroDes": "precioMetroDesMeanC"})

    x_train = x_train.merge(PMM, on='ciudad',how='left')
    x_test = x_test.merge(PMM, on='ciudad',how='left')
    
    
    
    #Calculo los mean target
    x_train_mean = mean_target_encoding(x_train, "provincia", "precio") 
    x_train_mean['precio'] = y_train
    x_train_mean = mean_target_encoding(x_train_mean, "tipodepropiedad", "precio")
    x_train_mean = mean_target_encoding(x_train_mean, "ciudad", "precio")
    x_train = mean_target_encoding(x_train_mean, "idzona", "precio")

    
    #Se los asigno a los test (NO LOS CALCULO CON ELLOS!!!!!!)
    x_test = mean_target_decoding(x_test, "provincia", x_train_mean)
    x_test = mean_target_decoding(x_test, "tipodepropiedad", x_train_mean)
    x_test = mean_target_decoding(x_test, "ciudad", x_train_mean)
    x_test = mean_target_decoding(x_test, "idzona", x_train_mean)
    
    backup = x_train_mean
    
    x_train = x_train_mean.drop(["precio","provincia","tipodepropiedad","ciudad"], axis=1)
    x_test.drop(["precio","provincia","tipodepropiedad", "ciudad"], axis=1, inplace = True)

    
    return x_train, x_test, y_train, y_test, backup

In [28]:
x_train, x_test, y_train, y_test, backup = armar_set_2(df)

y = y_train.values

x = x_train.values

Y_test = y_test.values

X_test = x_test.values

# divido el df 

/home/giaco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/giaco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/giaco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

### Creo LightGBM data containers

In [42]:
categorical_features = [c for c, col in enumerate(train.columns) if 'cat' in col]
train_data = lightgbm.Dataset(x, label=y, categorical_feature=categorical_features)
test_data = lightgbm.Dataset(X_test, label=Y_test)


In [43]:
#Lo entreno

params = {
    'objective': 'regression',
    'colsample_bytree': 0.9731668400523877,
    'learning_rate': 0.05,
    'min_child_samples': 180,#171 
    'min_child_weight': 1e-05, #1e-05
    'num_leaves': 35,#41 
    'reg_alpha': 10, #10
    'reg_lambda': 1000, #100
    'subsample': 0.9,#0.5575732396028996
    'max_depth':-1, 
    'random_state':314, 
    'silent':True,
    'max_bin': 800,
    'metric':'mae', 
    'n_jobs':None, 
    'n_estimators':100000
}

model = lightgbm.train(params,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=10000,
                       early_stopping_rounds=500)

/home/giaco/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/giaco/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/giaco/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/home/giaco/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:794: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[1]	valid_0's l1: 1.55475e+06
Training until validation scores don't improve for 500 rounds
[2]	valid_0's l1: 1.50475e+06
[3]	valid_0's l1: 1.4579e+06
[4]	valid_0's l1: 1.41317e+06
[5]	valid_0's l1: 1.37153e+06
[6]	valid_0's l1: 1.33184e+06
[7]	valid_0's l1: 1.29473e+06
[8]	valid_0's l1: 1.25928e+06
[9]	valid_0's l1: 1.22551e+06
[10]	valid_0's l1: 1.19409e+06
[11]	valid_0's l1: 1.16468e+06
[12]	valid_0's l1: 1.13578e+06
[13]	valid_0's l1: 1.10901e+06
[14]	valid_0's l1: 1.08435e+06
[15]	valid_0's l1: 1.06024e+06
[16]	valid_0's l1: 1.03733e+06
[17]	valid_0's l1: 1.01609e+06
[18]	valid_0's l1: 995687
[19]	valid_0's l1: 976583
[20]	valid_0's l1: 959019
[21]	valid_0's l1: 941524
[22]	valid_0's l1: 925208
[23]	valid_0's l1: 910090
[24]	valid_0's l1: 895178
[25]	valid_0's l1: 881402
[26]	valid_0's l1: 867792
[27]	valid_0's l1: 855516
[28]	valid_0's l1: 843235
[29]	valid_0's l1: 832108
[30]	valid_0's l1: 821258
[31]	valid_0's l1: 810465
[32]	valid_0's l1: 800669
[33]	valid_0's l1: 791370
[34]	

[305]	valid_0's l1: 557089
[306]	valid_0's l1: 556967
[307]	valid_0's l1: 556881
[308]	valid_0's l1: 556764
[309]	valid_0's l1: 556646
[310]	valid_0's l1: 556553
[311]	valid_0's l1: 556472
[312]	valid_0's l1: 556357
[313]	valid_0's l1: 556301
[314]	valid_0's l1: 556202
[315]	valid_0's l1: 556106
[316]	valid_0's l1: 556040
[317]	valid_0's l1: 555963
[318]	valid_0's l1: 555846
[319]	valid_0's l1: 555785
[320]	valid_0's l1: 555684
[321]	valid_0's l1: 555622
[322]	valid_0's l1: 555537
[323]	valid_0's l1: 555446
[324]	valid_0's l1: 555344
[325]	valid_0's l1: 555302
[326]	valid_0's l1: 555267
[327]	valid_0's l1: 555187
[328]	valid_0's l1: 555094
[329]	valid_0's l1: 554977
[330]	valid_0's l1: 554878
[331]	valid_0's l1: 554780
[332]	valid_0's l1: 554711
[333]	valid_0's l1: 554643
[334]	valid_0's l1: 554593
[335]	valid_0's l1: 554530
[336]	valid_0's l1: 554436
[337]	valid_0's l1: 554333
[338]	valid_0's l1: 554275
[339]	valid_0's l1: 554195
[340]	valid_0's l1: 554115
[341]	valid_0's l1: 554052
[

[609]	valid_0's l1: 540484
[610]	valid_0's l1: 540449
[611]	valid_0's l1: 540390
[612]	valid_0's l1: 540356
[613]	valid_0's l1: 540329
[614]	valid_0's l1: 540316
[615]	valid_0's l1: 540219
[616]	valid_0's l1: 540174
[617]	valid_0's l1: 540147
[618]	valid_0's l1: 540123
[619]	valid_0's l1: 540102
[620]	valid_0's l1: 540055
[621]	valid_0's l1: 540017
[622]	valid_0's l1: 539985
[623]	valid_0's l1: 539957
[624]	valid_0's l1: 539939
[625]	valid_0's l1: 539880
[626]	valid_0's l1: 539827
[627]	valid_0's l1: 539811
[628]	valid_0's l1: 539790
[629]	valid_0's l1: 539759
[630]	valid_0's l1: 539697
[631]	valid_0's l1: 539696
[632]	valid_0's l1: 539665
[633]	valid_0's l1: 539634
[634]	valid_0's l1: 539624
[635]	valid_0's l1: 539586
[636]	valid_0's l1: 539556
[637]	valid_0's l1: 539536
[638]	valid_0's l1: 539517
[639]	valid_0's l1: 539506
[640]	valid_0's l1: 539483
[641]	valid_0's l1: 539466
[642]	valid_0's l1: 539432
[643]	valid_0's l1: 539394
[644]	valid_0's l1: 539382
[645]	valid_0's l1: 539328
[

[913]	valid_0's l1: 533801
[914]	valid_0's l1: 533784
[915]	valid_0's l1: 533754
[916]	valid_0's l1: 533738
[917]	valid_0's l1: 533710
[918]	valid_0's l1: 533688
[919]	valid_0's l1: 533664
[920]	valid_0's l1: 533655
[921]	valid_0's l1: 533654
[922]	valid_0's l1: 533628
[923]	valid_0's l1: 533608
[924]	valid_0's l1: 533592
[925]	valid_0's l1: 533564
[926]	valid_0's l1: 533543
[927]	valid_0's l1: 533518
[928]	valid_0's l1: 533495
[929]	valid_0's l1: 533477
[930]	valid_0's l1: 533459
[931]	valid_0's l1: 533449
[932]	valid_0's l1: 533435
[933]	valid_0's l1: 533408
[934]	valid_0's l1: 533407
[935]	valid_0's l1: 533389
[936]	valid_0's l1: 533386
[937]	valid_0's l1: 533376
[938]	valid_0's l1: 533380
[939]	valid_0's l1: 533372
[940]	valid_0's l1: 533348
[941]	valid_0's l1: 533327
[942]	valid_0's l1: 533332
[943]	valid_0's l1: 533335
[944]	valid_0's l1: 533310
[945]	valid_0's l1: 533288
[946]	valid_0's l1: 533274
[947]	valid_0's l1: 533243
[948]	valid_0's l1: 533243
[949]	valid_0's l1: 533214
[

...

[12024]	valid_0's l1: 506398
[12025]	valid_0's l1: 506400
[12026]	valid_0's l1: 506399
[12027]	valid_0's l1: 506397
[12028]	valid_0's l1: 506396
[12029]	valid_0's l1: 506400
[12030]	valid_0's l1: 506398
[12031]	valid_0's l1: 506395
[12032]	valid_0's l1: 506399
[12033]	valid_0's l1: 506397
[12034]	valid_0's l1: 506398
[12035]	valid_0's l1: 506394
[12036]	valid_0's l1: 506393
[12037]	valid_0's l1: 506392
[12038]	valid_0's l1: 506390
[12039]	valid_0's l1: 506391
[12040]	valid_0's l1: 506391
[12041]	valid_0's l1: 506393
[12042]	valid_0's l1: 506393
[12043]	valid_0's l1: 506395
[12044]	valid_0's l1: 506388
[12045]	valid_0's l1: 506390
[12046]	valid_0's l1: 506388
[12047]	valid_0's l1: 506391
[12048]	valid_0's l1: 506390
[12049]	valid_0's l1: 506392
[12050]	valid_0's l1: 506391
[12051]	valid_0's l1: 506391
[12052]	valid_0's l1: 506392
[12053]	valid_0's l1: 506390
[12054]	valid_0's l1: 506393
[12055]	valid_0's l1: 506391
[12056]	valid_0's l1: 506394
[12057]	valid_0's l1: 506395
[12058]	valid_

[12307]	valid_0's l1: 506383
[12308]	valid_0's l1: 506380
[12309]	valid_0's l1: 506381
[12310]	valid_0's l1: 506380
[12311]	valid_0's l1: 506384
[12312]	valid_0's l1: 506386
[12313]	valid_0's l1: 506383
[12314]	valid_0's l1: 506381
[12315]	valid_0's l1: 506382
[12316]	valid_0's l1: 506383
[12317]	valid_0's l1: 506383
[12318]	valid_0's l1: 506383
[12319]	valid_0's l1: 506378
[12320]	valid_0's l1: 506377
[12321]	valid_0's l1: 506377
[12322]	valid_0's l1: 506380
[12323]	valid_0's l1: 506381
[12324]	valid_0's l1: 506376
[12325]	valid_0's l1: 506374
[12326]	valid_0's l1: 506370
[12327]	valid_0's l1: 506365
[12328]	valid_0's l1: 506366
[12329]	valid_0's l1: 506368
[12330]	valid_0's l1: 506368
[12331]	valid_0's l1: 506368
[12332]	valid_0's l1: 506368
[12333]	valid_0's l1: 506371
[12334]	valid_0's l1: 506371
[12335]	valid_0's l1: 506370
[12336]	valid_0's l1: 506371
[12337]	valid_0's l1: 506373
[12338]	valid_0's l1: 506370
[12339]	valid_0's l1: 506367
[12340]	valid_0's l1: 506368
[12341]	valid_

[12592]	valid_0's l1: 506384
[12593]	valid_0's l1: 506384
[12594]	valid_0's l1: 506386
[12595]	valid_0's l1: 506387
[12596]	valid_0's l1: 506388
[12597]	valid_0's l1: 506386
[12598]	valid_0's l1: 506385
[12599]	valid_0's l1: 506384
[12600]	valid_0's l1: 506383
[12601]	valid_0's l1: 506386
[12602]	valid_0's l1: 506384
[12603]	valid_0's l1: 506382
[12604]	valid_0's l1: 506380
[12605]	valid_0's l1: 506379
[12606]	valid_0's l1: 506381
[12607]	valid_0's l1: 506383
[12608]	valid_0's l1: 506382
[12609]	valid_0's l1: 506386
[12610]	valid_0's l1: 506387
[12611]	valid_0's l1: 506387
[12612]	valid_0's l1: 506385
[12613]	valid_0's l1: 506385
[12614]	valid_0's l1: 506383
[12615]	valid_0's l1: 506385
[12616]	valid_0's l1: 506386
[12617]	valid_0's l1: 506388
[12618]	valid_0's l1: 506390
[12619]	valid_0's l1: 506391
[12620]	valid_0's l1: 506387
[12621]	valid_0's l1: 506388
[12622]	valid_0's l1: 506388
[12623]	valid_0's l1: 506391
[12624]	valid_0's l1: 506391
[12625]	valid_0's l1: 506390
[12626]	valid_

[12878]	valid_0's l1: 506320
[12879]	valid_0's l1: 506321
[12880]	valid_0's l1: 506320
[12881]	valid_0's l1: 506315
[12882]	valid_0's l1: 506316
[12883]	valid_0's l1: 506318
[12884]	valid_0's l1: 506318
[12885]	valid_0's l1: 506317
[12886]	valid_0's l1: 506316
[12887]	valid_0's l1: 506312
[12888]	valid_0's l1: 506313
[12889]	valid_0's l1: 506311
[12890]	valid_0's l1: 506313
[12891]	valid_0's l1: 506313
[12892]	valid_0's l1: 506313
[12893]	valid_0's l1: 506311
[12894]	valid_0's l1: 506309
[12895]	valid_0's l1: 506309
[12896]	valid_0's l1: 506307
[12897]	valid_0's l1: 506309
[12898]	valid_0's l1: 506309
[12899]	valid_0's l1: 506306
[12900]	valid_0's l1: 506307
[12901]	valid_0's l1: 506305
[12902]	valid_0's l1: 506304
[12903]	valid_0's l1: 506304
[12904]	valid_0's l1: 506306
[12905]	valid_0's l1: 506310
[12906]	valid_0's l1: 506305
[12907]	valid_0's l1: 506305
[12908]	valid_0's l1: 506303
[12909]	valid_0's l1: 506305
[12910]	valid_0's l1: 506302
[12911]	valid_0's l1: 506298
[12912]	valid_

[13162]	valid_0's l1: 506331
[13163]	valid_0's l1: 506330
[13164]	valid_0's l1: 506331
[13165]	valid_0's l1: 506330
[13166]	valid_0's l1: 506334
[13167]	valid_0's l1: 506330
[13168]	valid_0's l1: 506328
[13169]	valid_0's l1: 506329
[13170]	valid_0's l1: 506328
[13171]	valid_0's l1: 506325
[13172]	valid_0's l1: 506324
[13173]	valid_0's l1: 506324
[13174]	valid_0's l1: 506327
[13175]	valid_0's l1: 506326
[13176]	valid_0's l1: 506323
[13177]	valid_0's l1: 506324
[13178]	valid_0's l1: 506325
[13179]	valid_0's l1: 506326
[13180]	valid_0's l1: 506325
[13181]	valid_0's l1: 506324
[13182]	valid_0's l1: 506325
[13183]	valid_0's l1: 506326
[13184]	valid_0's l1: 506326
[13185]	valid_0's l1: 506325
[13186]	valid_0's l1: 506325
[13187]	valid_0's l1: 506322
[13188]	valid_0's l1: 506324
[13189]	valid_0's l1: 506322
[13190]	valid_0's l1: 506323
[13191]	valid_0's l1: 506325
[13192]	valid_0's l1: 506329
[13193]	valid_0's l1: 506327
[13194]	valid_0's l1: 506326
[13195]	valid_0's l1: 506322
[13196]	valid_

In [44]:
prediccionLGBM = model.predict(x_test)

In [45]:
print(mean_absolute_error(y_test, prediccionLGBM))

506288.72457116836


# xgb

In [29]:
xgb_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, eval_metric='mae',
             gamma=0.015, importance_type='gain', learning_rate=0.025,
             max_delta_step=0, max_depth=11, min_child_weight=1, missing=None,
             n_estimators=3500, n_jobs=None, nthread=-1, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.733, verbosity=1)


#Fitteo
xgb_model.fit(x_train, y_train)


[21:34:39] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, eval_metric='mae',
             gamma=0.015, importance_type='gain', learning_rate=0.025,
             max_delta_step=0, max_depth=11, min_child_weight=1, missing=None,
             n_estimators=3500, n_jobs=None, nthread=-1, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.733, verbosity=1)

In [30]:
prediccion = xgb_model.predict(x_test)

In [31]:
#Estimo MAE
print(mean_absolute_error(y_test, prediccion))

480819.757765067


# Ensamble

In [135]:
ensamble = (prediccion*0.8 + prediccionLGBM*0.19)

In [136]:
#Estimo MAE
print(mean_absolute_error(y_test, ensamble))

479338.02631567966


In [137]:
480306.0866953195 #0.9 y 0.1
479338.02631567966 #0.8 y 0.19

479338.02631567966